# Test/Train split with year not working well for the share increase over year

In [ ]:
%pylab inline

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
df = pd.read_csv('day.csv',index_col='dteday', parse_dates=True)

In [ ]:
# logarithmic transformation of dependent cols
# (adding 1 first so that 0 values don't become -inf)
for col in ['casual', 'registered', 'cnt']:
    df['%s_log' % col] = np.log(df[col] + 1)

In [ ]:
#df['date'] = df.index.date
#df['yr'] = df.index.year
#df['season'] = df.index.quarter
#df['mnth'] = df.index.month
df['day'] = df.index.day
#df['weekday'] = df.index.dayofweek
df['weekofyear'] = df.index.weekofyear



In [ ]:
train_slicing = '2011'
test_slicing = '2012'

train = df[train_slicing]
test = df[test_slicing]

In [ ]:
by_season = train['cnt'].resample('Q').sum().plot(kind='barh')

In [ ]:
by_month = train['cnt'].resample('M').mean().plot(kind='barh')

In [ ]:
def get_rmsle(y_pred, y_actual):
    diff = np.log(y_pred + 1) - np.log(y_actual + 1)
    mean_error = np.square(diff).mean()
    return np.sqrt(mean_error)


def get_data():
    #avoid local variable name and upper leve variable name
    data = train
    return data


def custom_train_test_split(data, cutoff_day=15):
    train = data[data['day'] <= cutoff_day]
    test = data[data['day'] > cutoff_day]
    return train, test

def predict_on_validation_set(model, input_cols):
    #avoid local variable name and upper leve variable name
    data = get_data()

    train, test = custom_train_test_split(data)

    model_r = model.fit(train[input_cols], train['registered_log'])
    y_pred_r = np.exp(model_r.predict(test[input_cols])) - 1

    model_c = model.fit(train[input_cols], train['casual_log'])
    y_pred_c = np.exp(model_c.predict(test[input_cols])) - 1

    y_pred_comb = np.round(y_pred_r + y_pred_c)
    y_pred_comb[y_pred_comb < 0] = 0

    y_test_comb = test['cnt']

    score = get_rmsle(y_pred_comb, y_test_comb)
    return (y_pred_comb, y_test_comb, score)

# predict on test set & transform output back from log scale
def predict_on_test_set(model, x_cols):
    casual_model = model.fit(train[x_cols], train['casual_log'])
    y_pred_cas = casual_model.predict(test[x_cols])
    y_pred_cas = np.exp(y_pred_cas) - 1
    registered_model = model.fit(train[x_cols], train['registered_log'])
    y_pred_reg = registered_model.predict(test[x_cols])
    y_pred_reg = np.exp(y_pred_reg) - 1
    # add casual & registered predictions together
    return y_pred_cas + y_pred_reg

In [ ]:
params = {'n_estimators': 1000, 'max_depth': 15, 'random_state': 0, 'min_samples_split' : 5, 'n_jobs': -1}
rf_model = RandomForestRegressor(**params)
rf_cols = [
    'weathersit', 'temp', 'atemp', 'windspeed',
    'workingday', 'season', 'holiday', 
     'weekday', 'weekofyear'
    #,'peak','sticky','hour',
    ]

(rf_p, rf_t, rf_score) = predict_on_validation_set(rf_model, rf_cols)
print (rf_score)

In [ ]:
df[rf_cols].corr()

In [ ]:
params = {'n_estimators': 150, 'max_depth': 5, 'random_state': 0, 'min_samples_leaf' : 10, 'learning_rate': 0.1, 'subsample': 0.7, 'loss': 'ls'}
gbm_model = GradientBoostingRegressor(**params)
gbm_cols = [
    'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
    'holiday', 'workingday', 'season',
     'weekday'
    #, 'ideal', 'count_season','hour','yr'
]

(gbm_p, gbm_t, gbm_score) = predict_on_validation_set(gbm_model, gbm_cols)
print (gbm_score)

In [ ]:
df[gbm_cols].corr()

In [ ]:
# the blend gives a better score on the leaderboard, even though it does not on the validation set
y_p = np.round(.2*rf_p + .8*gbm_p)
print (get_rmsle(y_p, rf_t))

In [ ]:
pd.options.mode.chained_assignment = None
rf_pred = predict_on_test_set(rf_model, rf_cols)
gbm_pred = predict_on_test_set(gbm_model, gbm_cols)
y_pred = np.round(0.2*rf_pred + 0.8*gbm_pred)
# output predictions for submission
test['pred_cnt'] = y_pred
reslut = test[['cnt','pred_cnt']]
#test.to_csv('submit.csv', index=True)

In [ ]:
reslut